In [24]:
from scipy.io import loadmat
from sklearn.preprocessing import MinMaxScaler
from scipy import signal
import numpy as np
import math
import os
from scipy.stats import kurtosis

def calculate_time_domain_features(segment):
    mean_value = np.mean(segment)
    std_dev = np.std(segment)
    var = np.var(segment, ddof=1)
    kurt = kurtosis(segment)
    #max_value = np.max(segment)
    return mean_value, std_dev, var, kurt

def load_data():
    data_dir = "E:/STUDY/Publications/Thesis/Brain Emotion Detection/Dataset/Practice with datasets/Seed/Preprocessed_EEG/"
    fs = 200
    fStart = [1, 4, 8, 14, 31]
    fEnd = [3, 7, 13, 30, 50]
    all_channels = range(62)

    filename_label = "label"
    label = loadmat(data_dir + filename_label)
    label = label["label"][0]

    datasets_X, datasets_y = [], []

    for filename_data in os.listdir(data_dir):
        if filename_data in ["label.mat", "readme.txt"]:
            continue

        data_all = loadmat(data_dir + filename_data)
        scenes = list(data_all.keys())[3:]

        for index, scene in enumerate(scenes):
            dataset_X = []
            data = data_all[scene][:, :37000]  # Considering all channels

            for band_index, band in enumerate(fStart):
                b, a = signal.butter(4, [fStart[band_index]/fs, fEnd[band_index]/fs], 'bandpass')
                filtedData = signal.filtfilt(b, a, data)
                features = []

                for lead in all_channels:
                    feature = []
                    for de_index in range(0, filtedData.shape[1] - fs, fs):
                        segment = filtedData[lead, de_index: de_index + fs]
                        # Calculate time domain features
                        mean_value, std_dev, var, kurt = calculate_time_domain_features(segment)
                        # Append features to the list
                        feature.append([mean_value, std_dev, var, kurt])
                    features.append(feature)

                features = np.array(features)
                dataset_X.append(features)

            dataset_X = np.array(dataset_X)
            dataset_X = dataset_X.reshape((dataset_X.shape[0], dataset_X.shape[1], -1))
            
            datasets_X.append(dataset_X)
            datasets_y.append(label[index])

    datasets_X, datasets_y = np.array(datasets_X), np.array(datasets_y)

    return datasets_X, datasets_y


datasets_X, datasets_y = load_data()
print(datasets_X.shape)
print(datasets_y.shape)


(15, 5, 62, 736)
(15,)


In [ ]:
new_datasets_X = np.copy(datasets_X)
new_datasets_X = np.transpose(new_datasets_X, (0, 2, 3, 1))
print(new_datasets_X.shape)

In [ ]:
np.save('TimeDomain_Feature_X.npy', new_datasets_X)
np.save('TimeDomain_Feature_y.npy', datasets_y)